### Here a CNN with 3 different conv layers  is implemented. Each conv layer is followed by a maxpool layer. At the end two different fully connected layers are also considered.

In [1]:
import tensorflow as tf
from util.data import *

#### Data and Parameters -------------------------------------------------------------------------

In [3]:
Sample_number=10000
Feature_Number=1024
batch_size=256
learning_rate=0.01
display_step=20
epochs=100
DataObj=Data(data_len=Feature_Number,Sample_Number=Sample_number,Test_percentage=40)
Train_x,Train_y,Test_x,Test_y=DataObj.CNN_data()
Number_of_batches=int(len(Train_x)/batch_size)

Train_x=Train_x.reshape([-1,32,32,1]) # 32*32 =1024
Test_x=Test_x.reshape([-1,32,32,1])

print (Train_x.shape)
print (Test_x.shape)

X=tf.placeholder("float",[None,32,32,1])
Y=tf.placeholder("float",[None,10])

p_keep_conv = tf.placeholder("float")
p_keep_FC = tf.placeholder("float")

Parameters={'Wc1':tf.Variable(tf.random_normal([2,2,1,32],stddev=0.01)),
           'Wc2':tf.Variable(tf.random_normal([3,3,32,64],stddev=0.01)),
           'Wc3':tf.Variable(tf.random_normal([4,4,64,128],stddev=0.01)),
           'WF1':tf.Variable(tf.random_normal([4*4*128,700],stddev=0.01)),
           'WF2':tf.Variable(tf.random_normal([700,800],stddev=0.01)),
           'Out':tf.Variable(tf.random_normal([800,10],stddev=0.01)),
           'bc1':tf.Variable(tf.zeros([32])),
           'bc2':tf.Variable(tf.zeros([64])),
           'bc3':tf.Variable(tf.zeros([128])),
           'bF1':tf.Variable(tf.zeros([700])),
           'bF2':tf.Variable(tf.zeros([800])),
           'bO':tf.Variable(tf.zeros([10]))}

(6000, 32, 32, 1)
(4000, 32, 32, 1)


#### Model definition ----------------------------------------------------------------------------

#### Graph definition---------------------------------------------------------------------------------------------

In [ ]:
def model(Input,Weights):
    
    conv1=tf.nn.conv2d(Input,Weights['Wc1'],strides=[1,1,1,1],padding='SAME')
    conv1=tf.nn.bias_add(conv1,Weights['bc1'])
    conv1=tf.nn.relu(conv1) # conv1 shape=[-1,32,32,32]
    Mpl1=tf.nn.max_pool(conv1,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')
    Mpl1=tf.nn.dropout(Mpl1,p_keep_conv) # Mpl1 shape= [-1,16,16,32]
    
    conv2=tf.nn.conv2d(Mpl1,Weights['Wc2'],strides=[1,1,1,1],padding='SAME')
    conv2=tf.nn.bias_add(conv2,Weights['bc2'])
    conv2=tf.nn.relu(conv2) # conv2 shape=[-1,16,16,64]
    Mpl2=tf.nn.max_pool(conv2,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')
    Mpl2=tf.nn.dropout(Mpl2,p_keep_conv) #[Mpl2 shape=[-1,8,8,64]]
    
    conv3=tf.nn.conv2d(Mpl2,Weights['Wc3'],strides=[1,1,1,1],padding='SAME')
    conv3=tf.nn.bias_add(conv3,Weights['bc3'])
    conv3=tf.nn.relu(conv3)    # conv3 shape=[-1,8,8,128]
    Mpl3=tf.nn.max_pool(conv3,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')
    Mpl3=tf.nn.dropout(Mpl3,p_keep_conv) # Mpl3 shape=[-1,4,4,128]
    
    # below section is important 
    InF1=tf.reshape(Mpl3,[-1,Weights['WF1'].get_shape().as_list()[0]])
    InF1=tf.nn.dropout(InF1,p_keep_FC)
    
    FC1=tf.nn.relu(tf.add(tf.matmul(InF1,Weights['WF1']),Weights['bF1']))
    
    
    InF2=tf.nn.dropout(FC1,p_keep_FC)
    FC2=tf.nn.relu(tf.add(tf.matmul(InF2,Weights['WF2']),Weights['bF2']))
    
    InOut=tf.nn.dropout(FC2,p_keep_FC)
    
    Out=tf.nn.relu(tf.add(tf.matmul(InOut,Weights['Out']),Weights['bO']))
    
    
    return tf.nn.softmax(Out)

In [ ]:
Prediction_prob=model(X,Parameters)

cost=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=Prediction_prob,labels=Y))

train_step=tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

True_Predictions=tf.equal(tf.argmax(Prediction_prob,1),tf.argmax(Y,1))

Accuracy=tf.reduce_mean(tf.cast(True_Predictions,'float'))

##### Graph runinng ----------------------------------------------------------------------------------------

In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for i in range(epochs):
        avg_cost=0
        for j in range(Number_of_batches):
            Trx,Try=next_batch(batch_size, Train_x,Train_y)
            sess.run(train_step,feed_dict={X:Trx,Y:Try,p_keep_conv:0.8,p_keep_FC:0.8})
            avg_cost+=sess.run(cost,feed_dict={X:Trx,Y:Try,p_keep_conv:1.,p_keep_FC:1.})/Number_of_batches
        if i%display_step==0:
            Train_Accuracy=sess.run(Accuracy,feed_dict={X:Trx,Y:Try,p_keep_conv:1.,p_keep_FC:1.})
            Test_Accuracy=sess.run(Accuracy,feed_dict={X:Test_x,Y:Test_y,p_keep_conv:1.,p_keep_FC:1.})
            print ("Epoch: %03d/%03d cost: %.9f train_acc: %.3f test_acc: %.3f" 
                   % (i, epochs, avg_cost, Train_Accuracy, Test_Accuracy))
sess.close()